**Import necessary libraries**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Load the images to be aligned**

In [ ]:
img1 = cv2.imread('images/exp4-2.jpg')
img2 = cv2.imread('images/exp4-1.jpg')

**Display the images**

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)
plt.title('Image 1')
plt.subplot(122)
plt.imshow(img2)
plt.title('Image 2')
plt.show()

**Get human-guided control points**

In [ ]:
# Get the number of control points to be selected
num_control_points = 4

# Initialize the control points list
control_points = []

# Create a window to display the image
plt.imshow(img2)

# Print the control points
print(control_points)
# Iterate over the number of control points
for i in range(num_control_points):
    # Get the mouse click coordinates
    x, y = plt.ginput(1)[0]

    # Add the control point to the list
    control_points.append((x, y))

# Display the control points on the images
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)
plt.title('Image 1 with control points')
plt.scatter(control_points[:, 0], control_points[:, 1], c='red', marker='o')
plt.subplot(122)
plt.imshow(img2)
plt.title('Image 2 with control points')
plt.scatter(control_points[:, 0], control_points[:, 1], c='red', marker='o')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to compute the mean squared error (MSE) between two images
def compute_mse(img1, img2):
    mse = np.mean((img1 - img2)**2)
    return mse

# Function to warp an image to another image using a homography matrix
def warp_image(img, H, output_shape):
    warped_img = cv2.warpPerspective(img, H, output_shape)
    return warped_img

# Function to evaluate the efficacy of human-guided control point selection for image alignment
def evaluate_control_point_selection(img1, img2, control_points):
    # Compute the homography matrix
    H = cv2.findHomography(control_points[:, ::-1], control_points[:, ::-1])[0]

    # Warp the second image to the first image
    warped_img2 = warp_image(img2, H, img1.shape)

    # Compute the MSE between the aligned images
    mse = compute_mse(img1, warped_img2)

    return mse

# Load the images to be aligned
img1 = cv2.imread('images/exp4-2.jpg')
img2 = cv2.imread('images/exp4-1.jpg')

# Display the images to the user
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)
plt.title('Image 1')
plt.subplot(122)
plt.imshow(img2)
plt.title('Image 2')
plt.show()

# Get human-guided control points
num_control_points = 4

# Initialize the control points list
control_points = []

# Iterate over the number of control points
for i in range(num_control_points):
    # Get the mouse click coordinates
    x, y = plt.ginput(i)

    # Add the control point to the list
    control_points.append((x, y))

# Display the control points on the images
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)
plt.title('Image 1 with control points')
plt.scatter(control_points[:, 0], control_points[:, 1], c='red', marker='o')
plt.subplot(122)
plt.imshow(img2)
title('Image 2 with control points')
plt.scatter(control_points[:, 0], control_points[:, 1], c='red', marker='o')
plt.show()

# Evaluate the efficacy of human-guided control point selection
mse = evaluate_control_point_selection(img1, img2, control_points)

# Print the MSE
print('MSE: ', mse)

# Display the aligned images
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)
plt.title('Image 1')
plt.subplot(122)
plt.imshow(warped_img2)
plt.title('Warped Image 2')
plt.show()

In [ ]:
import cv2

# Load source and target images
sourceImage = cv2.imread('images/exp4-2.jpg')
targetImage = cv2.imread('images/exp4-1.jpg')

# Display source and target images
cv2.imshow("Source Image", sourceImage)
cv2.imshow("Target Image", targetImage)
cv2.waitKey(0)

# Prompt user to select control points
sourcePoints = cv2.selectPoints(sourceImage, "Select control points in source image")
targetPoints = cv2.selectPoints(targetImage, "Select control points in target image")

# Perform image alignment
alignedImage = cv2.warpPerspective(sourceImage, cv2.findHomography(sourcePoints, targetPoints), targetImage.shape[:2])

# Display aligned image
cv2.imshow("Aligned Image", alignedImage)
cv2.waitKey(0)

# Evaluate alignment quality
alignmentQuality = cv2.matchTemplate(alignedImage, targetImage, cv2.TM_SQDIFF_NORM)

# Calculate efficacy of human-guided control point selection
efficacy = alignmentQuality / targetImage.size

# Output evaluation results and efficacy metrics
print("Alignment quality:", alignmentQuality)
print("Efficacy:", efficacy)


In [ ]:
from __future__ import print_function
import cv2
import numpy as np
 
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
 
def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
 
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
 
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
 
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
 
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
 
  return im1Reg, h
 
if __name__ == '__main__':
 
  # Read reference image
  refFilename = "images/exp4-1.png"
  print("Reading reference image : ", refFilename)
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
 
  # Read image to be aligned
  imFilename = "images/exp4-2.png"
  print("Reading image to align : ", imFilename);
  im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
 
  print("Aligning images ...")
  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
  imReg, h = alignImages(im, imReference)
 
  # Write aligned image to disk.
  outFilename = "aligned.jpg"
  print("Saving aligned image : ", outFilename);
  cv2.imwrite(outFilename, imReg)
 
  # Print estimated homography
  print("Estimated homography : \n",  h)

In [ ]:
# Import the necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the images to be aligned
im1 = cv2.imread('images/exp4-1.png')
im2 = cv2.imread('images/exp4-2.png')

# Convert images to grayscale
im1_gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
im2_gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

# Display the images
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(im1_gray, cmap='gray')
plt.title('Image 1')
plt.subplot(122)
plt.imshow(im2_gray, cmap='gray')
plt.title('Image 2')
plt.show()

# Ask the user to select four corresponding points on each image
print('Please select four corresponding points on each image by clicking on them.')
points1 = [] # List to store the points on image 1
points2 = [] # List to store the points on image 2

# Define a callback function to record the mouse clicks
def mouse_callback(event, x, y, flags, param):
    global points1, points2
    if event == cv2.EVENT_LBUTTONDOWN: # Left mouse button clicked
        if len(points1) < 4: # Still need to select points on image 1
            points1.append((x, y)) # Append the point to the list
            cv2.circle(im1, (x, y), 5, (0, 0, 255), -1) # Draw a red circle on the image
            cv2.imshow('Image 1', im1) # Show the updated image
        elif len(points2) < 4: # Still need to select points on image 2
            points2.append((x, y)) # Append the point to the list
            cv2.circle(im2, (x, y), 5, (0, 0, 255), -1) # Draw a red circle on the image
            cv2.imshow('Image 2', im2) # Show the updated image

# Create windows to display the images
cv2.namedWindow('Image 1')
cv2.namedWindow('Image 2')

# Set the mouse callback function for the windows
cv2.setMouseCallback('Image 1', mouse_callback)
cv2.setMouseCallback('Image 2', mouse_callback)

# Show the images
cv2.imshow('Image 1', im1)
cv2.imshow('Image 2', im2)

# Wait for the user to press any key
cv2.waitKey(0)

# Destroy all windows
cv2.destroyAllWindows()

# Convert the points to numpy arrays
points1 = np.array(points1)
points2 = np.array(points2)

# Print the selected points
print('The selected points on image 1 are:')
print(points1)
print('The selected points on image 2 are:')
print(points2)

# Find the homography matrix that maps the points from image 1 to image 2
H, _ = cv2.findHomography(points1, points2)

# Apply the homography matrix to warp image 1 to align with image 2
im1_aligned = cv2.warpPerspective(im1, H, (im2.shape[1], im2.shape[0]))

# Display the aligned image
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(im1_aligned, cv2.COLOR_BGR2RGB))
plt.title('Aligned Image')
plt.show()

# Compute the mean squared error (MSE) between the aligned image and image 2
mse = np.mean((im1_aligned - im2) ** 2)
print(f'The mean squared error between the aligned image and image 2 is: {mse:.4f}')

# Compute the structural similarity index measure (SSIM) between the aligned image and image 2
ssim = cv2.compare_ssim(im1_aligned, im2, multichannel=True)
print(f'The structural similarity index measure between the aligned image and image 2 is: {ssim:.4f}')